<center><h1>Lab 1: Optical flow</h1></center>
<center>Darius Fang Cmput 487 2023W</center>
<center>Python 3.8.3</center>
<br>
<font color="#FF0000">
<!-- Demo: In your lab session during the week following the due date. -->
</font>
<h2>Lab #1.1</h2>
<b><font size="+1">Exercise 1 (3):</font></b> <font size="+1">Image differences</font>
<p><font size="+1">a (1). Capture images</font>
This is the initial part of the report. It is to capture frames and save frames into a folder. The default frame size is 640x480. 
The example I have below saves the files in 'captureImage', consisting of panning my camera across my plushie. The video only pans left right.

In [ ]:
import cv2
import numpy as np

#rgb to bgr
import matplotlib.pyplot as plt
# the line below is used for in the lab
# cam = cv2.VideoCapture(0, cv2.CAP_FIREWIRE)
# the line below is to capture a normal webcam
cam = cv2.VideoCapture(0)
cv2.namedWindow("test")
# path file to be saved
path = "captureImage/"
img_counter = 0
while True:
    # read frame
    ret, frame = cam.read()
    if not ret:
        print("failed to grab frame")
        break
    cv2.imshow("test", frame)
    k = cv2.waitKey(1)
    if k%256 == 27:
    # ASCII:ESC pressed, exit
        print("Escape hit, closing...")
        break
    elif k%256 == 32:
    # ASCII:SPACE pressed, capture frame
        img_name = path + "opencv_frame_{}.png".format(img_counter)
        cv2.imwrite(img_name, frame)
        # print("{} written!".format(img_name))
        img_counter += 1
# only prints how many frames were saved
print("{} written!".format(img_name))
cam.release()
cv2.destroyAllWindows()

<br><br><br><br>
</p><p><font size="+1">b (2). Image differences</font>

The "delta image" is a crucial aspect of the optic flow function. It is determined by calculating the difference between two images. 
    $$ \frac {\delta Im}{\delta t} = Im(t+1) - Im(t)$$
    It is important to convert the numpy arrays into float data type to avoid any loss of data due to negative numbers and overflow issues. A threshold is set to eliminate image noises and possible light or background noise between the two frames. In the code, a threshold of 10 is used, and any absolute difference found to be less than 10 is set to 0. Note that negative values in the delta image correspond to the opposite direction of the optic flow vector.

The example provided only demonstrates the calculation for the first 9 iterations.
<br>
<br>
Input of 2 images
<br>
<img src="captureImage/opencv_frame_8.png"  width="500"> <img src="captureImage/opencv_frame_9.png"  width="500">

<br>
Output of the difference (note that this is from opencv imshow, and shows only uint8 type)
<br>
<img src="replaceImage/opencv_difference_9.png"  width="500">

In [ ]:
import cv2
import numpy as np
path = "captureImage/"
path1 = "replaceImage/"
ImageShape = [480,640]
imageWidth = ImageShape[1]
imageHeight = ImageShape[0]
size = (imageWidth, imageHeight)
for i in range(10):
    #read 2 images
    img_nameNew = path + "opencv_frame_{}.png".format(i+1)
    img_nameOld = path + "opencv_frame_{}.png".format(i)
    Inew = cv2. imread(img_nameNew,0)
    Iold = cv2. imread(img_nameOld,0) 
    cv2.imshow("test1", Inew)
    cv2.imshow("test2", Iold)
    #convert to float
    Inew = Inew.astype(np.float)
    Iold = Iold.astype(np.float)
    # take  difference
    dIm = np.array(Inew) - np.array(Iold)
  

    while True:
        #Threshold
        dIm[np.absolute(dIm)<10] = 0
        cv2.imshow("test", dIm.astype(np.float))
        k = cv2.waitKey(1)
        if k%256 == 27:
            break
    cv2.imwrite(path1 + "opencv_difference_{}.png".format(i), dIm)
cv2.destroyAllWindows()



<b><font size="+1">Exercise 2 (7):</font></b> <font size="+1">Optical flow</font> 

In the next step, the computation of the optic flow will be based on user input. For each identified blob in the image, the difference between the two images and the gradient is calculated to determine the vector flow. The vectors are then displayed as an overlay on the image using a quiver plot. It is important to note that changing the block size can affect the calculated optic flow vectors. Sampling a small section will only show movement for small pixel changes, while larger block sizes may result in misleading optic flow errors due to certain conditions. For example, when two objects are moving in different directions (which has not been tested in this case). It is also important to note that 2D optic flow does not solve the aperture problem, which results in an ambiguous determination of the optic flow.

2D optic Flow formula:

$$ u = M\backslash  dIm, \\ M = \begin{bmatrix} . & . \\ 
                                                . & . \\ 
                                                \frac {\delta Im}{\delta t} & \frac {\delta Im}{\delta t} \\
                                                . & . \\ 
                                                . & . \\ \end{bmatrix}, 
                            dIm = \begin{bmatrix} . \\ 
                                                .  \\ 
                                                -\frac {\delta Im}{\delta t} \\
                                                .  \\ 
                                                .  \\ \end{bmatrix},
                            u = \begin{bmatrix} \delta x \\
                                                 \delta y
                                               \end{bmatrix},$$
To solve $u$ we have to use the least squares method provided by numpy 

Update January 31st: There was an error in the orientation of the u and v values, as they were switched. This was due to MATLAB outputting in the format col, row, while numpy outputs in row, col. All of the code for lab 1.2 has been updated to correct this mistake. As a reference, the old tracking code can be found in the previous version and is labeled as "output_previous (u,v) are flipped".

An example with 64 block size:

<img src="report Images/outputOptic64_1.jpg"  width="500"> <img src="report Images/outputOptic64_2.jpg"  width="500">


The user inputs the block size by entering a single integer. To input a block size of 2x2, the user should enter '2'.

The output will be under 'outputOptic\<BlockSize>.mp4'


 

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
# %matplotlib inline
blockSize = int(input("Specify block size: "))
path = "captureImage/"
ImageShape = [480,640]
imageWidth = ImageShape[1]
imageHeight = ImageShape[0]
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
result = cv2.VideoWriter('outputOptic' + str(blockSize) + '.mp4', fourcc, 20, (imageWidth, imageHeight))
# result.release()
for index in range(115):
    img_nameNew = path + "opencv_frame_{}.png".format(index+1)
    img_nameOld = path + "opencv_frame_{}.png".format(index)
    Inew = cv2. imread(img_nameNew,0)
    Iold = cv2. imread(img_nameOld,0) 
    size = (imageWidth, imageHeight)

    Inew = Inew.astype(np.float)
    Iold = Iold.astype(np.float)
    # difference image  

    dIm = np.array(Inew) - np.array(Iold)
    dIm = dIm.astype(np.float)
    dIm[np.absolute(dIm)<10 ] = 0
    ###
    ### UPDATE Jan 31
    ### dVM and dUm were swaped since np gradient does [rows, col] while in MATLAB it does [col, row]
    ###
    [dVm, dUm] = np.gradient(Iold)
    #row, column -> y, x
    X = range(1+int(blockSize/2),imageWidth,blockSize)
    Y = range(1+int(blockSize/2),imageHeight,blockSize)
    U = np.ones((len(Y),len(X)))
    V = np.ones((len(Y),len(X)))
    for i in range(len(X)):
        for j in range(len(Y)):
            # get frame
            k = j*blockSize
            l = i*blockSize
            #get blob
            dI = dIm[k:k+blockSize, l:l+blockSize]
            dU = dUm[k:k+blockSize, l:l+blockSize]
            dV = dVm[k:k+blockSize, l:l+blockSize]
            # vector, flatten matrix
            dIv = dI.flatten()
            dUv = dU.flatten()
            dVv = dV.flatten()
            dIv = np.array([dIv])
            #least squares
            M = np.array([dUv, dVv])
            dIv = dIv.T
            M = M.T
            arr = np.linalg.lstsq(M,dIv, rcond=None)[0]
            # normalise
            u, v = -arr / np.linalg.norm(arr)
            #x motion
            U[j,i] = u
            #y motion
            V[j,i] = v

    [X,Y] = np.meshgrid(X,Y)
    Iold = cv2.imread(path + "opencv_frame_{}.png".format(index))
    Iold = cv2.cvtColor(Iold, cv2.COLOR_BGR2RGB)
    # matplotlib configerations to remove border
    fig = plt.figure(frameon = False)
    fig.set_size_inches(16, 12)
    ax = plt.Axes(fig, [0., 0., 1., 1.], )
    ax.set_axis_off()
    fig.add_axes(ax)
    plt.quiver(X,Y,U,V, minlength=4)
    ax.imshow(Iold)
    fig.canvas.draw()
    # matplotlib fig to numpy
    image_from_plot = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    image_from_plot = image_from_plot.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    image_from_plot = cv2.cvtColor(image_from_plot, cv2.COLOR_BGR2RGB)
    image_from_plot = cv2.resize(image_from_plot, (640,480), interpolation = cv2.INTER_AREA)
    # write
    result.write(image_from_plot)
    plt.clf()
    plt.close("all")
print("finished")
result.release()
cv2.destroyAllWindows


<ipython-input-1-29198896480f>:57: RuntimeWarning: invalid value encountered in true_divide
  u, v = -arr / np.linalg.norm(arr)


finished


<function destroyAllWindows>



<h2>Lab 1.2: 2D Video tracking</h2>


<br>
<br>
<b><font size="+1">Exercise 1 (5):</font></b>
<font size="+1">Translational x-y tracking </font>

By leveraging our understanding of optic flow, we will use it to track the movement of a single blob specified by the user. The user will provide an initial position $p$ by drawing it on the screen, along with the frame width and height. For each subsequent optic flow calculation, the position $p$ will be updated based on the calculated vector flow.


It is important to note that while this 2D implementation can effectively track an object moving left-right or up-down, it can be challenging to track an object that rotates or undergoes an affine transformation. Scaling the object is not possible using a single vector flow, as scaling is similar to a zoom in/out vector field. Image warping is required to handle affine transformations, and updating the width and height of the image.


To maintain precision, the global position p is saved as a float, as the change in the optic flow can sometimes result in fractional numbers. However, for display purposes, the pixels are rounded to an integer. This implementation does not incorporate any image warping, and instead uses registration-based tracking, which is precise but can fail if the object is lost.

$$ p = p+u$$

An example:

<img src="report Images/outputTrack2_2.jpg"  width="500"> <img src="report Images/outputTrack2_1.jpg"  width="500">


The user can define the target object by creating a rectangular region of interest on the video frame using a mouse drag. Upon pressing 'esc', the processing of the video will commence. The output will be under 'outputTrack2.mp4'



In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
# %matplotlib inline
blockSize = 16
path = "captureImage/"
ImageShape = [480,640]
imageWidth = ImageShape[1]
imageHeight = ImageShape[0]
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
result = cv2.VideoWriter('outputTrack2.mp4', fourcc, 20, (imageWidth, imageHeight))
# result.release()
drawing = False
ix,iy = -1,-1
width, height = 0,0
def draw_rectangle(event, x, y, flags, param):
   global ix, iy, drawing,  width, height
   if event == cv2.EVENT_LBUTTONDOWN:
      drawing = True
      ix = x
      iy = y
   elif event == cv2.EVENT_LBUTTONUP:
      drawing = False
      width = x- ix
      height = y- iy
      print(width, height)
      cv2.rectangle(Input, (ix, iy),(ix + width, iy + height),(0, 255, 255),4)

img_nameOld = path + "opencv_frame_{}.png".format(15)
Input = cv2. imread(img_nameOld) 
cv2.namedWindow("test")
cv2.setMouseCallback("test", draw_rectangle)


while True:
    # dIm[dIm>=80] = 255
    cv2.imshow("test", Input)
    k = cv2.waitKey(1)
    if k%256 == 27:
        break
cv2.destroyAllWindows
for index in range(15,115):
    img_nameNew = path + "opencv_frame_{}.png".format(index+1)
    img_nameOld = path + "opencv_frame_{}.png".format(index)
    Inew = cv2. imread(img_nameNew,0)
    Iold = cv2. imread(img_nameOld,0) 
    size = (imageWidth, imageHeight)

    Inew = Inew.astype(np.float)
    Iold = Iold.astype(np.float)
    # difference image  
    dIm = np.array(Inew) - np.array(Iold)
    dIm[np.absolute(dIm)<10 ] = 0
    Iold = Iold.astype(np.float)

    [dVm, dUm] = np.gradient(Iold)
    #row, column -> y, x
    intX = int(ix)
    intY = int(iy)
    dI = dIm[intY:intY+height, intX:intX+width]
    dU = dUm[intY:intY+height, intX:intX+width]
    dV = dVm[intY:intY+height, intX:intX+width]
    # vector, flatten matrix
    dIv = dI.flatten()
    dUv = dU.flatten()
    dVv = dV.flatten()
    dIv = np.array([dIv])
    #least squares
    M = np.array([dUv, dVv])
    dIv = dIv.T
    M = M.T
    # print(np.linalg.lstsq(M,dIv, rcond=None))
    arr = np.linalg.lstsq(M,dIv, rcond=None)[0]
    u, v = -arr 

    ix = ix +  u
    #y motion
    iy = iy +  v
    #print(u,v)
    Iold = cv2.imread(path + "opencv_frame_{}.png".format(index+1))
    cv2.rectangle(Iold, (intX, intY),(intX + width, intY + height),(0, 255, 255),4)
    # write
    result.write(Iold)
print("finished")
result.release()
cv2.destroyAllWindows


<b><font size="+1">Exercise 2:</font></b> <font size="+1">Pyramidal tracking</font>

Pyramidal tracking provides a more accurate tracking solution compared to the previous implementation. By creating a pyramid of the image through down-sampling and Gaussian filtering, the change in vectors are then propagated, with the highest level having the greatest influence. The user inputs the number of levels and the resizing factor for the pyramid. For example, with 2 levels and a resize of 2, the influence of the top level would be the vector multiplied by $2^2$. This results in a smoother and more precise tracking solution.

Update on Jan 31: guassian filter only works with odd integers like 1x1, 3x3 etc. So I also added Pydown to do 1/2 sampling. If any other value is specified it will also do 1/2 sampling. The output of the video is similar to x-y translation, only the translations look smoother due to pyramid giving a better percision than only have 1 level.

Example:

<img src="report Images/outputTrack3_2.jpg"  width="500"> <img src="report Images/outputTrack3_1.jpg"  width="500">


The user can specify the number of levels and resize factor for the pyramidal tracking system. They can interact with the program by drawing a rectangular region on the frame using their mouse. To initiate the processing of the video, the user simply presses the 'esc' key.

The output will be under 'outputTrack3.mp4'

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# %matplotlib inline
blockSize = 16
path = "captureImage/"
ImageShape = [480,640]
imageWidth = ImageShape[1]
imageHeight = ImageShape[0]
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
result = cv2.VideoWriter('outputTrack3.mp4', fourcc, 20, (imageWidth, imageHeight))
# result.release()
drawing = False
ix,iy = -1,-1
width, height = 0,0
level = int(input("input level:"))
resizeFactor = int(input("input resizing factor: "))
def draw_rectangle(event, x, y, flags, param):
   global ix, iy, drawing,  width, height
   if event == cv2.EVENT_LBUTTONDOWN:
      drawing = True
      ix = x
      iy = y
   elif event == cv2.EVENT_LBUTTONUP:
      drawing = False
      width = x- ix
      height = y- iy
      print(ix, iy, width, height)
      cv2.rectangle(Input, (ix, iy),(ix + width, iy + height),(0, 255, 255),4)
      cv2.destroyAllWindows

img_nameOld = path + "opencv_frame_{}.png".format(15)
Input = cv2. imread(img_nameOld) 
cv2.namedWindow("test")
cv2.setMouseCallback("test", draw_rectangle)

# draw rectangle from user, then press esc
while True:
    cv2.imshow("test", Input)
    k = cv2.waitKey(1)
    if k%256 == 27:
        break
cv2.destroyAllWindows
for index in range(15,115):
    img_nameNew = path + "opencv_frame_{}.png".format(index+1)
    img_nameOld = path + "opencv_frame_{}.png".format(index)
    Inew = cv2. imread(img_nameNew,0)
    Iold = cv2. imread(img_nameOld,0)
    size = (imageWidth, imageHeight)
    #pyramid
    G = Inew.copy()
    gpA = [G]
    kernel_size = (resizeFactor, resizeFactor)
    for i in range(level):
        # Get the new dimensions of the image
        rows, cols = G.shape[:2]
        new_rows, new_cols = int(rows/resizeFactor), int(cols/resizeFactor)
        ###
        ### Update Jan 31
        ### Pydown was added to do resize factor of 2. 4, 16, etc were not implemented though. any odd resize factor will be assumed to work
        ###
        # Downsample the image
        if (resizeFactor%2 == 1):
            G = cv2.GaussianBlur(G, kernel_size, 0)
            G = cv2.resize(G, (new_cols, new_rows), cv2.INTER_AREA)
        else:
            G = cv2.pyrDown(G)
        gpA.append(G)
    F = Iold.copy()
    gpB = [F]
    for i in range(level):
        rows, cols = F.shape[:2]
        new_rows, new_cols = int(rows/resizeFactor), int(cols/resizeFactor)
        # Downsample the image
        if (resizeFactor%2 == 1):
            F = cv2.GaussianBlur(F, kernel_size, 0)
            F = cv2.resize(F, (new_cols, new_rows), cv2.INTER_AREA)
        else:
            F = cv2.pyrDown(F)
        gpB.append(F)
    # pyramid updated length and widths
    width = width/pow(resizeFactor, level)
    height = height/pow(resizeFactor, level)
    # pyramid point positon
    ix = ix/pow(resizeFactor, level)
    iy = iy/pow(resizeFactor,level)
    #pyramid for loop
    for i in range(level, -1, -1):
        # final vector to be added to global point p
        subU = 0
        subV = 0
        Inew = gpA[i].astype(np.float)
        Iold = gpB[i].astype(np.float)
        # difference image  
        dIm = np.array(Inew) - np.array(Iold)
        dIm[np.absolute(dIm)<10 ] = 0
        [dVm, dUm] = np.gradient(Iold)
        #row, column -> y, x
        intX = int(ix)
        intY = int(iy)
        Intwidth = int(width)
        IntHeight = int(height)
        # specified blob
        dI = dIm[intY:intY+IntHeight, intX:intX+Intwidth]
        dU = dUm[intY:intY+IntHeight, intX:intX+Intwidth]
        dV = dVm[intY:intY+IntHeight, intX:intX+Intwidth]
        # vector, flatten matrix
        dIv = dI.flatten()
        dUv = dU.flatten()
        dVv = dV.flatten()
        dIv = np.array([dIv])
        #least squares
        M = np.array([dUv, dVv])
        dIv = dIv.T
        M = M.T
        arr = np.linalg.lstsq(M,dIv, rcond=None)[0]
        u, v = -arr 
        
        subU = subU +  u*pow(resizeFactor, i)
        #y motion
        subV = subV +  v*(pow(resizeFactor,i))
        Iold = Iold.astype(np.uint8)
        # print(subU, subV)
        cv2.rectangle(Iold, (intX, intY),(intX + Intwidth, intY + IntHeight),(0, 255, 255),4)
        # while True:
        #     # dIm[dIm>=80] = 255
        #     cv2.imshow("test", Iold)
        #     k = cv2.waitKey(1)
        #     if k%256 == 27:
        #         break
        # cv2.destroyAllWindows
        if (i == 0):
            Iold = cv2.imread(path + "opencv_frame_{}.png".format(index+1))
            cv2.rectangle(Iold, (intX, intY),(intX + Intwidth, intY + IntHeight),(0, 255, 255),4)
            # write
            result.write(Iold)
            ix = ix + subU
            iy = iy + subV
        else:
            #update the local pyramid variables
            ix = ix*resizeFactor
            iy = iy*resizeFactor
            width = width*resizeFactor
            height = height*resizeFactor

    
print("finished")
result.release()
cv2.destroyAllWindows